In [ ]:
import cv2
import numpy as np

In [ ]:
img = cv2.imread(r"D:\bharat\Images\mikebirdy.jpg")
# print(img)
# img = cv2.resize(img, (500,600))


mask1 = np.zeros(img.shape[:2],np.uint8)
# print(mask1)
bgmask = np.zeros((1,65),np.float64)
fgmask = np.zeros((1,65),np.float64) 


r = (396,501,3534,2462)
cv2.grabCut(img,mask1,r,bgmask,fgmask,10,cv2.GC_INIT_WITH_RECT)

mask2 =np.where((mask1 == 2)|(mask1 == 0),0,1).astype("uint8")
print(mask2)
img = img*mask2[:,:,np.newaxis]
print(img)
cv2.imshow("window",img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
import cv2 
import numpy as np

In [11]:
cap = cv2.VideoCapture(r"D:\bharat\Images\Dance.mp4")
# two algorithm for remove video background
algo1 =cv2.createBackgroundSubtractorKNN(detectShadows = True)
algo2 = cv2.createBackgroundSubtractorMOG2(detectShadows = True)

while True:
    r,f = cap.read()
    if r == True:
        f = cv2.resize(f,(400,400))
        r1 = algo1.apply(f)
        r2 = algo2.apply(f)
        cv2.imshow("video",f)
        cv2.imshow("algo1",r1)
        cv2.imshow("algo2",r2)
        if cv2.waitKey(40) & 0xff == ord("q"):
            break
    else:
        break
cap.release()
cv2.destroyAllWindows()


# Object Tracking And Detection

In [22]:
import cv2
import numpy as np

In [23]:
cap = cv2.VideoCapture(r"D:\bharat\Images\Dance.mp4")
r,f = cap.read()
x,y,w,h = 611,78,110,292 

t = (x,y,w,h)
roi = f[y:y+h,x:x+w]

hsv_roi = cv2.cvtColor(roi,cv2.COLOR_BGR2HSV)
print(hsv_roi)

mask = cv2.inRange(hsv_roi,np.array((0.,60.,32.)),np.array((180.,255.,255.)))


roi_hist = cv2.calcHist([hsv_roi],[0],mask,[180],[0,180])
cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)
cv2.imshow("test",roi)
while cap.isOpened():
    r,f = cap.read()
    if r== True:
#         f = cv2.resize(f,(400,500))
#         cv2.imshow("video",f)
        
        if cv2.waitKey(25) & 0xff ==ord("q"):
            break
            
    else:
        break
        
cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [25]:
print(hsv_roi)

NameError: name 'hsv_roi' is not defined

In [31]:
import cv2
import numpy as np

cap = cv2.VideoCapture(r"D:\bharat\Images\Dance.mp4")
ret, frame = cap.read()

x, y, w, h = 600, 78, 110, 292
track_window = (x, y, w, h)
roi = frame[y:y+h, x:x+w]

hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv_roi, np.array((0., 60., 32.)), np.array((180., 255., 255.)))

roi_hist = cv2.calcHist([hsv_roi], [0], mask, [180], [0, 180])
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)

cv2.imshow("ROI", roi)

term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

while cap.isOpened():
    ret, frame = cap.read()
    if ret:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)
        
        ret, track_window = cv2.meanShift(dst, track_window, term_crit)
        
        x, y, w, h = track_window
        result = cv2.rectangle(frame, (x, y), (x+w, y+h), 255, 2)
        
        cv2.imshow("Tracking", result)
        
        if cv2.waitKey(25) & 0xff == ord("q"):
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()


error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [27]:
t

(611, 78, 110, 292)

In [28]:
roi

array([], shape=(292, 0, 3), dtype=uint8)

In [18]:
import cv2
import numpy as np

# Open the video file
cap = cv2.VideoCapture(r"D:\bharat\Images\Dance.mp4")
ret, frame = cap.read()

# Define the initial region of interest (ROI)
x, y, w, h = 611, 78, 110, 292
track_window = (x, y, w, h)

# Set up the ROI for tracking
roi = frame[y:y+h, x:x+w]
hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv_roi, np.array((0., 60., 32.)), np.array((180., 255., 255.)))
roi_hist = cv2.calcHist([hsv_roi], [0], mask, [180], [0, 180])
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)

# Setup the termination criteria, either 10 iterations or move by at least 1 pt
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

while cap.isOpened():
    ret, frame = cap.read()
    if ret:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)

        # Apply CamShift to get the new location
        ret, track_window = cv2.CamShift(dst, track_window, term_crit)

        # Draw it on the image
        pts = cv2.boxPoints(ret)
        pts = np.int0(pts)
        img2 = cv2.polylines(frame, [pts], True, 255, 2)

        cv2.imshow('Tracking', img2)

        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()


error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [29]:
roi

array([], shape=(292, 0, 3), dtype=uint8)